In [ ]:
import pandas as pd
import numpy as np

import sys
sys.path.append('..')
from autoencoder import ToyDetectors, DEDXEncoder

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import tensorflow.keras.backend as K

from __future__ import absolute_import ,division ,print_function
from scipy.optimize import curve_fit
import tensorflow as tf
import tensorflow_probability as tfp
import matplotlib.pyplot as plt


tfd = tfp.distributions
tfe = tf.contrib.eager
tfb = tfp.bijectors


print(tf.__version__)
print(tfp.__version__)

In [ ]:
TD = ToyDetectors()
DFTD = TD.GenerateToyParticles().sample(frac=1)

In [ ]:
DFTD.head()

In [ ]:
signal = DFTD.query("particle!=1")[["TPCROC0"]]

In [ ]:
p = DFTD.query("particle!=1")[["p"]]

In [ ]:
signalnp = np.array(signal)
pnp = np.array(p)

In [ ]:
plt.hist(signalnp, bins = 100)
plt.show()

In [ ]:
plt.hist(pnp, bins = 100)
plt.show()

In [ ]:
def BetheBlochAleph(bg,kp1,kp2,kp3,kp4,kp5):# ,kp1=0.76176e-1,kp2=10.632,kp3=0.13279e-4,kp4=1.8631,kp5=1.9479):
            beta = bg/tf.sqrt(1.+ bg*bg)
            aa   = tf.exp(kp4*tf.log(beta))
            bb   = tf.exp(-kp5*tf.log(bg))
            bb   = tf.log(kp3+bb)
            return (kp2-aa-bb)*kp1/aa

In [ ]:
mass_e = 0.000511
mass_pi = 0.139    
mass_K = 0.494
mass_p = 0.938

In [ ]:
def BetheBlochAlephNP(bg,kp1=0.76176e-1,kp2=10.632,kp3=0.13279e-4,kp4=1.8631,kp5=1.9479):
        beta = bg/np.sqrt(1.+ bg*bg)
        aa   = np.exp(kp4*np.log(beta))
        bb   = np.exp(-kp5*np.log(bg))
        bb   = np.log(kp3+bb)
        return (kp2-aa-bb)*kp1/aa

In [ ]:
x = pnp
y = signalnp

In [ ]:
t = np.arange(0.1, 10., 0.1)
plt.plot(t, BetheBlochAlephNP(t/mass_pi),t , BetheBlochAlephNP(t/mass_e))

In [ ]:
def log_prob(x, y, kp1, kp2, kp3, kp4, kp5): 
    pi_mean = BetheBlochAleph(x/mass_pi,kp1,kp2,kp3,kp4,kp5)
    e_mean = BetheBlochAleph(x/mass_e,kp1,kp2,kp3,kp4,kp5) 
    p_mean = BetheBlochAleph(x/mass_p,kp1,kp2,kp3,kp4,kp5) 
    K_mean = BetheBlochAleph(x/mass_K,kp1,kp2,kp3,kp4,kp5)
    
    pi_sigma=0.1*pi_mean
    e_sigma =0.1*e_mean
    p_sigma =0.1*p_mean
    K_sigma =0.1*K_mean
    
    p_pi    = tfd.Normal(loc=pi_mean,scale=pi_sigma).prob(y)
    p_e     = tfd.Normal(loc= e_mean,scale= e_sigma).prob(y)
    p_p     = tfd.Normal(loc= p_mean,scale= p_sigma).prob(y)
    p_K     = tfd.Normal(loc= K_mean,scale= K_sigma).prob(y)
    
    
    lnp = tf.log((p_pi   + p_e + p_p + p_K)/4.)
    
    extraloss = - 1e3*(kp3-0.13279e-4)**2
    return tf.reduce_mean(lnp) + extraloss


unnormalized_posterior_log_prob = lambda *args: log_prob(x,y, *args)

In [ ]:
kp1 = tf.Variable(0.7 ,dtype=tf.float64,name="kp1") # 0.76176e-1
kp2 = tf.Variable(12.0,dtype=tf.float64,name="kp2") # 10.632
kp3 = tf.Variable(0.13279e-4,dtype=tf.float64,name="kp3")
kp4 = tf.Variable(2.0,dtype=tf.float64,name="kp4") # 1.8631
kp5 = tf.Variable(2.2,dtype=tf.float64,name="kp5") # 1.9479




In [ ]:
loss=-unnormalized_posterior_log_prob(kp1,kp2,tf.sqrt(kp3**2),kp4,kp5)
optimizer=tf.train.AdagradOptimizer(learning_rate=0.01)
trainer  =optimizer.minimize(loss,var_list=[kp1,kp2,kp3,kp4,kp5])

In [ ]:
#init =tf.global_variables_initializer() 
#with tf.Session() as session:
#    session.run(init)
#    print(session.run(loss))

In [ ]:
init =tf.global_variables_initializer() 
with tf.Session() as session:
    session.run(init)
    for _ in range(5000):
        l1=session.run(loss)
        session.run(trainer)
        l2=session.run(loss)
        kp1_v=session.run(kp1)
        kp2_v=session.run(kp2)
        kp3_v=np.abs(session.run(kp3))
        kp4_v=session.run(kp4)
        kp5_v=session.run(kp5)
        
        
        print("loss (prev): ",l1,"loss (post): ",l2)
        print(kp1_v,kp2_v,kp3_v,kp4_v,kp5_v)
        print()


In [ ]:
out = pd.DataFrame([kp1_v, kp2_v, kp3_v, kp4_v, kp5_v])

In [ ]:
out.transpose().to_csv("MCMCresults.csv")